In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.0f}'.format

In [2]:
nurse_df = pd.read_csv('https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/cdph-skilled-nursing-totals.csv')
nurse_df.head(5)

,date,staff_active_cases,patients_active_cases,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,staff_deaths_note
0,2021-11-28,202,202,55419,65333,251,9350,NaN
1,2021-11-27,187,196,55401,65320,251,9350,NaN
2,2021-11-26,202,201,55387,65294,251,9349,NaN
3,2021-11-25,186,215,55373,65276,251,9349,NaN
4,2021-11-24,206,221,55354,65250,251,9348,NaN


In [3]:
nurse_df.drop(nurse_df.columns[[1, 2, 7]], axis=1, inplace=True)
nurse_df['type'] = 'SN'
nurse_df

,date,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type
0,2021-11-28,55419,65333,251,9350,SN
1,2021-11-27,55401,65320,251,9350,SN
2,2021-11-26,55387,65294,251,9349,SN
3,2021-11-25,55373,65276,251,9349,SN
4,2021-11-24,55354,65250,251,9348,SN
...,...,...,...,...,...,...
539,2020-04-29,2594,3923,19,618,SN
540,2020-04-28,2323,3515,1,578,SN
541,2020-04-25,2329,3441,1,545,SN
542,2020-04-24,2099,3141,1,486,SN


In [4]:
today = nurse_df[(nurse_df['date'] == '2021-11-28')]
today.set_index('date', inplace=True)
today

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type
date,,,,,
2021-11-28,55419,65333,251,9350,SN


In [5]:
senior_df = pd.read_csv('https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/cdph-adult-and-senior-care-totals.csv')

In [6]:
senior_df

,date,confirmed_cases,deaths,active_cases,source_url
0,2021-07-27,43066,3749,307,https://www.cdss.ca.gov/Portals/9/Additional-R...
1,2021-07-14,42910,3747,181,https://www.cdss.ca.gov/Portals/9/Additional-R...
2,2021-07-12,42811,3746,97,https://www.cdss.ca.gov/Portals/9/Additional-R...
3,2021-06-21,42733,3744,59,https://www.cdss.ca.gov/Portals/9/Additional-R...
4,2021-05-31,42696,3744,89,https://www.cdss.ca.gov/Portals/9/Additional-R...
...,...,...,...,...,...
290,2020-05-02,1404,175,nan,https://www.cdss.ca.gov/Portals/9/Additional-R...
291,2020-05-01,1340,170,nan,https://www.cdss.ca.gov/Portals/9/Additional-R...
292,2020-04-29,1267,155,nan,https://www.cdss.ca.gov/Portals/9/Additional-R...
293,2020-04-28,1198,147,nan,https://www.cdss.ca.gov/Portals/9/Additional-R...


In [7]:
senior_df.drop(senior_df.columns[[3, 4]], axis=1, inplace=True)
senior_df['type'] = 'SC'
sc_today = senior_df[(senior_df['date'] == '2021-07-27')]
sc_today

,date,confirmed_cases,deaths,type
0,2021-07-27,43066,3749,SC


In [8]:
sc_today.set_index('date', inplace=True)
sc_today

,confirmed_cases,deaths,type
date,,,
2021-07-27,43066,3749,SC


In [9]:
latest_df = pd.merge(today, sc_today, left_on=['date'], right_on=['date'])
latest_df

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type_x,confirmed_cases,deaths,type_y
date,,,,,,,,


In [10]:
combined = today.merge(sc_today, how='outer', left_index=True, right_index=True)
combined

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type_x,confirmed_cases,deaths,type_y
date,,,,,,,,
2021-07-27,nan,nan,nan,nan,NaN,43066,3749,SC
2021-11-28,55419,65333,251,9350,SN,nan,nan,NaN


In [11]:
combined['patient_cases'] = combined["patients_confirmed_cases"] + combined["confirmed_cases"]
combined['patient_deaths'] = combined["patients_deaths"] + combined["deaths"]
combined

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type_x,confirmed_cases,deaths,type_y,patient_cases,patient_deaths
date,,,,,,,,,,
2021-07-27,nan,nan,nan,nan,NaN,43066,3749,SC,nan,nan
2021-11-28,55419,65333,251,9350,SN,nan,nan,NaN,nan,nan


In [12]:
combined2 = combined[['patient_cases', 'patient_deaths', 'staff_confirmed_cases', 'staff_deaths']]
combined2

,patient_cases,patient_deaths,staff_confirmed_cases,staff_deaths
date,,,,
2021-07-27,nan,nan,nan,nan
2021-11-28,nan,nan,55419,251
